<a href="https://colab.research.google.com/github/SundasKhanT/urdu-multilingual-open-llms-evaluation/blob/main/Qwen2_5_0_5B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets scikit-learn torch --quiet

In [2]:
from datasets import load_dataset
dataset = load_dataset("AhmadMustafa/Urdu-Instruct-News-Category-Classification")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/105M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/11.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100674 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11187 [00:00<?, ? examples/s]

In [3]:
import torch

# Empty the GPU cache
torch.cuda.empty_cache()

# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")

# Move the model to GPU with half-precision
model.half().to("cuda")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((896,), eps=1e-06)
    (rotary_emb): Qwen2

In [46]:
def get_model_output(prompt = "", max_new_tokens = 128):
    urdu_inputs = tokenizer.encode(prompt, return_tensors="pt")
    urdu_inputs = urdu_inputs.to("cuda")
    urdu_outputs = model.generate(urdu_inputs, max_new_tokens = max_new_tokens)

    output = tokenizer.decode(urdu_outputs[0])
    cleaned_output = output.replace("", "").replace("", "").strip()

    prompt_response = "" + cleaned_output

    return prompt_response

In [47]:
torch.cuda.empty_cache()

In [48]:
import pandas as pd
from datasets import load_dataset

# Load the dataset
dataset = load_dataset('AhmadMustafa/Urdu-Instruct-News-Category-Classification')

# Check available splits
print(dataset)  # Shows all available splits like 'train', 'test', 'validation', etc.

# Convert to pandas dataframe (adjust split name if needed)
df = dataset['test'].to_pandas()  # or 'test_2' if it exists

# Display the first rows
df.head()


DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets', 'template_id', 'template_lang'],
        num_rows: 100674
    })
    test: Dataset({
        features: ['inputs', 'targets', 'template_id', 'template_lang'],
        num_rows: 11187
    })
})


,inputs,targets,template_id,template_lang
0,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",2,[urd]
1,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd]
2,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""سائنس اور ٹیکنالوجی"" سے تعلق ر...",2,[urd]
3,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd]
4,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd]


In [39]:
zero_shot = """
آپ کو ایک اردو نیوز آرٹیکل دیا گیا ہے۔
آپ کا کام اس آرٹیکل کو اس کے مواد کے مطابق پہلے سے طے شدہ زمرے میں تقسیم کرنا ہے۔

پہلے سے طے شدہ زمرے یہ ہیں:
۱. تفریح
۲. سائنس اور ٹیکنالوجی
۳. کھیل
۴. پاکستان
۵. طرزِ زندگی
۶. کاروبار و معیشت
۷. بین الاقوامی
۸. دلچسپ و عجیب
۹. صحت
"""


In [40]:
three_shot = """
آپ کو ایک اردو نیوز آرٹیکل دیا گیا ہے۔
آپ کا کام اس آرٹیکل کو اس کے مواد کے مطابق پہلے سے طے شدہ زمرے میں تقسیم کرنا ہے۔

پہلے سے طے شدہ زمرے یہ ہیں:
۱. پاکستان
۲. کھیل
۳. سائنس اور ٹیکنالوجی
۴. تفریح
۵. بین الاقوامی
۶. کاروبار و معیشت
۷. طرزِ زندگی
۸. صحت
۹. دلچسپ و عجیب

### مثال ۱
آرٹیکل:
'اسلام آباد میں آج نیشنل ہاکی چیمپیئن شپ کے فائنل میں پاکستان ٹیم نے بھارت کو شکست دی۔'
پریڈکٹڈ زمرہ: 'کھیل'

### مثال ۲
آرٹیکل:
'وفاقی حکومت نے نئی معاشی پالیسی کا اعلان کیا ہے جس سے چھوٹے کاروباری حضرات مستفید ہوں گے۔'
پریڈکٹڈ زمرہ: 'کاروبار و معیشت'

### مثال ۳
آرٹیکل:
'جدید تحقیق کے مطابق مریخ پر پانی کے آثار ملے ہیں جو خلائی سائنس میں ایک بڑی پیش رفت ہے۔'
پریڈکٹڈ زمرہ: 'سائنس اور ٹیکنالوجی'

### آپ کا کام:
آرٹیکل: "{article_text}"
پریڈکٹڈ زمرہ:
"""

In [41]:
chain_of_thought_urdu_news = """
آپ کو ایک اردو نیوز آرٹیکل کا ڈیٹاسیٹ دیا گیا ہے۔ آپ کا کام ہر آرٹیکل کو اس کے مواد کے مطابق پہلے سے طے شدہ زمروں میں سے ایک میں درجہ بندی کرنا ہے۔

پہلے سے طے شدہ زمرے ہیں:
۱. پاکستان
۲. کھیل
۳. سائنس اور ٹیکنالوجی
۴. تفریح
۵. بین الاقوامی
۶. کاروبار و معیشت
۷. طرزِ زندگی
۸. صحت
۹. دلچسپ و عجیب

### غور و فکر کے مراحل:

### مرحلہ ۱: آرٹیکل کا مواد پڑھیں اور سمجھیں
آرٹیکل کے جملے اور بنیادی موضوع کو پڑھیں، اور اس کی نوعیت کا اندازہ لگائیں۔

### مرحلہ ۲: بنیادی تھیم کا تعین
آرٹیکل کے اہم موضوع کا فیصلہ کریں۔ یہ عنوان اور پہلے چند جملے پڑھ کر سمجھا جا سکتا ہے۔

### مرحلہ ۳: زمرے سے ملاپ
آرٹیکل کے مرکزی تھیم کے مطابق فہرست میں سے مناسب زمرہ منتخب کریں۔

### مثال ۱
آرٹیکل:
'گزشتہ رات لاہور میں کرکٹ کے ایک میچ کے دوران ٹیم نے شاندار کارکردگی دکھائی۔'

#### سوچ:
- بنیادی مواد: لاہور میں کرکٹ میچ
- مرکزی تھیم: کھیل
- زمرہ سے ملاپ: کھیل

پریڈکٹیڈ زمرہ: 'کھیل'

### مثال ۲
آرٹیکل:
'وفاقی وزیر خزانہ نے روپے کی قدر میں بہتری اور اسٹاک مارکیٹ میں تیزی کے حوالے سے بیان جاری کیا۔'

#### سوچ:
- بنیادی مواد: معاشی صورتحال اور اسٹاک مارکیٹ
- مرکزی تھیم: معیشت اور کاروبار
- زمرہ سے ملاپ: کاروبار و معیشت

پریڈکٹیڈ زمرہ: 'کاروبار و معیشت'

### مثال ۳
آرٹیکل:
'طبی ماہرین کے مطابق روزانہ آٹھ گلاس پانی پینا انسانی جسم کے مدافعت نظام کے لیے انتہائی ضروری ہے۔'

#### سوچ:
- بنیادی مواد: انسانی صحت اور پانی کے فوائد
- مرکزی تھیم: صحت
- زمرہ سے ملاپ: صحت

پریڈکٹیڈ زمرہ: 'صحت'

### آپ کا کام:
آرٹیکل: "{article_text}"
پریڈکٹیڈ زمرہ:
"""

In [42]:
# This matches the EXACT labels inside the AhmadMustafa News Dataset
category_translation = {
    'پاکستان': 'پاکستان',
    'کھیل': 'کھیل',
    'سائنس اور ٹیکنالوجی': 'سائنس اور ٹیکنالوجی',
    'تفریح': 'تفریح',
    'بین الاقوامی': 'بین الاقوامی',
    'کاروبار و معیشت': 'کاروبار و معیشت',
    'طرزِ زندگی': 'طرزِ زندگی',
    'صحت': 'صحت',
    'دلچسپ و عجیب': 'دلچسپ و عجیب'
}

In [44]:
import time
import os
from tqdm.notebook import tqdm
import pandas as pd

# 1. Prepare the subset (using the first 100 for your evaluation)
df_subset = df.iloc[:100].copy()

# 2. Progress Management: Load existing results if they exist
file_path = 'model_zs_classification_results.parquet'
if os.path.exists(file_path):
    df_subset = pd.read_parquet(file_path)
    # Count how many rows are NOT Null to find where to start
    start_index = df_subset['predicted_category'].notna().sum()
else:
    df_subset['predicted_category'] = None
    start_index = 0

print(f"Starting from index: {start_index}")

# 3. The Inference Loop
for i in tqdm(range(start_index, len(df_subset))):
    news = df_subset.iloc[i]['inputs']

    # f-string ensures \n is handled as a character, not literal text
    # Standardizing on 'پریڈکٹیڈ زمرہ:'
    prompt_input = f"{zero_shot}\n\nمضمون کا مواد:\n{news}\n\nپریڈکٹیڈ زمرہ:"

    try:
        # .strip() is CRITICAL here to remove \n from the model output
        raw_output = get_model_output(prompt_input, max_new_tokens=15)
        clean_output = raw_output.strip()

        df_subset.at[df_subset.index[i], 'predicted_category'] = clean_output

    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    # Save progress every 10 rows to prevent data loss
    if (i + 1) % 10 == 0:
        df_subset.to_parquet(file_path)

# Final Save
df_subset.to_parquet(file_path)
print("Inference complete! File saved as model_zs_classification_results.parquet")

Starting from index: 100


0it [00:00, ?it/s]

Inference complete! File saved as model_zs_classification_results.parquet


In [45]:
zs_results = pd.read_parquet("model_zs_classification_results.parquet")
zs_results.head()

,inputs,targets,template_id,template_lang,predicted_category
0,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",2,[urd],آپ کو ایک اردو نیوز آر
1,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],آپ کا کام اس آرٹیکل ک
2,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""سائنس اور ٹیکنالوجی"" سے تعلق ر...",2,[urd],آپ کا کام اس آرٹیکل ک
3,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],آپ کو ایک اردو نیوز آر
4,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],آپ کو ایک اردو نیوز آر


In [35]:
zs_results.tail()

,inputs,targets,template_id,template_lang,predicted_category
95,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""کھیل"" سے تعلق رکھتی ہیں",2,[urd],آپ کا کام اس آرٹیکل ک
96,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],آپ کا یہ میں ہوتا
97,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],آپ کا کام اس آرٹیکل ک
98,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",2,[urd],دیے گی خبروں کو ایک
99,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""کاروبار و معیشت"" سے تعلق رکھتی...",2,[urd],آپ کو ایک اردو نیوز آر


In [49]:
import time
from tqdm.notebook import tqdm

# Ensure 'predicted_category' column exists
df['predicted_category'] = None

# Use only the first 100 rows
df_subset = df.iloc[:100].copy()

# Try to load progress from previous Parquet file
try:
    df_progress = pd.read_parquet('model_ts_classification_results.parquet')
    last_idx = df_progress['predicted_category'].last_valid_index()
    start_index = last_idx + 1 if last_idx is not None else 0
except (FileNotFoundError, ValueError):
    df_progress = df_subset.copy()
    start_index = 0

# Loop over the subset
for i, row in tqdm(df_subset.iloc[start_index:].iterrows(), initial=start_index, total=len(df_subset) - start_index):
    news = row['inputs'] # Corrected column name to 'inputs'
    label = row['targets'] # Corrected column name to 'targets'
    label_urdu = category_translation.get(label, label)  # Translated label if available

    # prompt_input = three_shot + " مضمون کا مواد: " + news + "\nپیش گوئی شدہ زمرہ:"
    prompt_input = three_shot.format(article_text=news)


    try:
        df_subset.at[i, 'predicted_category'] = get_model_output(prompt_input, max_new_tokens=128)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    # Save progress every 50 rows (or any interval)
    if (i + 1) % 50 == 0:
        df_subset.to_parquet('model_ts_classification_results.parquet')

# Save final results
df_subset.to_parquet('model_ts_classification_results.parquet')

  0%|          | 0/100 [00:00<?, ?it/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [50]:
ts_results = pd.read_parquet("model_ts_classification_results.parquet")
ts_results.head()

,inputs,targets,template_id,template_lang,predicted_category
0,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",2,[urd],آپ کو ایک اردو نیوز آرٹیکل دیا گیا ہے۔\nآپ کا ...
1,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],آپ کو ایک اردو نیوز آرٹیکل دیا گیا ہے۔\nآپ کا ...
2,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""سائنس اور ٹیکنالوجی"" سے تعلق ر...",2,[urd],آپ کو ایک اردو نیوز آرٹیکل دیا گیا ہے۔\nآپ کا ...
3,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],آپ کو ایک اردو نیوز آرٹیکل دیا گیا ہے۔\nآپ کا ...
4,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],آپ کو ایک اردو نیوز آرٹیکل دیا گیا ہے۔\nآپ کا ...


In [53]:
import time
from tqdm.notebook import tqdm
import pandas as pd

# Ensure 'predicted_category' column exists
df['predicted_category'] = None

# Use only the first 100 rows
df_subset = df.iloc[:100].copy()

# Try to load progress from previous Parquet file
try:
    df_progress = pd.read_parquet('model_cot_classification_results.parquet')
    last_idx = df_progress['predicted_category'].last_valid_index()
    start_index = last_idx + 1 if last_idx is not None else 0
except (FileNotFoundError, ValueError):
    df_progress = df_subset.copy()
    start_index = 0

# Loop over the subset
for i, row in tqdm(df_subset.iloc[start_index:].iterrows(), initial=start_index, total=len(df_subset) - start_index):
    news = row['inputs']   # Column with Urdu text
    label = row['targets'] # Column with actual label
    label_bn = label       # No translation needed for Urdu

    # Prepare prompt for CoT classification
    prompt_input = chain_of_thought_urdu_news + " مضمون کا مواد: " + news

    try:
        df_subset.at[i, 'predicted_category'] = get_model_output(prompt_input, max_new_tokens=256)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    # Save progress every 50 rows
    if (i + 1) % 50 == 0:
        df_subset.to_parquet('model_cot_classification_results.parquet')

# Save final results
df_subset.to_parquet('model_cot_classification_results.parquet')

  0%|          | 0/100 [00:00<?, ?it/s]

In [54]:
cot_results = pd.read_parquet("model_cot_classification_results.parquet")
cot_results.tail()

,inputs,targets,template_id,template_lang,predicted_category
95,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""کھیل"" سے تعلق رکھتی ہیں",2,[urd],آپ کو ایک اردو نیوز آرٹیکل کا ڈیٹاسیٹ دیا گیا ...
96,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],آپ کو ایک اردو نیوز آرٹیکل کا ڈیٹاسیٹ دیا گیا ...
97,اس پیراگراف (خبروں) کو مندرجہ ذیل درجوں (کیٹگ...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",1,[urd],آپ کو ایک اردو نیوز آرٹیکل کا ڈیٹاسیٹ دیا گیا ...
98,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""تفریح"" سے تعلق رکھتی ہیں",2,[urd],آپ کو ایک اردو نیوز آرٹیکل کا ڈیٹاسیٹ دیا گیا ...
99,دیے گی خبروں کو ایک لفظ یا درجہ (کیٹگری) میں ب...,"یہ دی گی خبریں ""کاروبار و معیشت"" سے تعلق رکھتی...",2,[urd],آپ کو ایک اردو نیوز آرٹیکل کا ڈیٹاسیٹ دیا گیا ...


In [55]:
def clean_predicted_category(text, category_translation):
    if text is None:
        return ""

    import re
    # Look for the last occurrence of "پیش گوئی شدہ زمرہ:"
    # This assumes the model's output appends the category after this phrase.
    match = re.search(r'پیش گوئی شدہ زمرہ:\s*(.*?)(?:\n|$)', text, re.DOTALL)
    if match:
        potential_category_text = match.group(1).strip()
    else:
        # Fallback if "پیش گوئی شدہ زمرہ:" is not found, which shouldn't happen
        # if `get_model_output` returns the full prompt.
        potential_category_text = text.strip()

    # Remove any unwanted tokens like <pad>, </s>, etc.
    potential_category_text = potential_category_text.replace("<pad>", "").replace("</s>", "").strip()

    # First, check for exact matches with or without bolding
    # Check if any category is within bold tags, e.g., **کھیل**
    bold_match = re.search(r'\*\*([^\*]+)\*\*', potential_category_text)
    if bold_match:
        extracted_category = bold_match.group(1).strip()
        if extracted_category in category_translation.values():
            return extracted_category

    # If no bold match, or not in category_translation, check for direct presence
    for category in category_translation.values():
        # Exact match
        if category == potential_category_text:
            return category
        # Or if the model outputs more, like "کھیل\n\nExplanation"
        if potential_category_text.startswith(category):
            return category

    # If nothing explicitly matches after cleaning, try to find direct substring matches
    # (less precise, but might catch variations)
    for category in category_translation.values():
        if category in potential_category_text:
            return category

    # If no category is found, return None
    return None

In [58]:
clean_predicted_category("یہ مضمون تعلیم کے زمرے میں آتا ہے۔ \n\n...", category_translation)

In [57]:
clean_predicted_category("پیش گوئی شدہ زمرہ: **کھیل** \n\nیہ مضمون.....", category_translation)


'کھیل'

In [59]:
clean_predicted_category(None, category_translation)

''

In [60]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelBinarizer

# --- Robust extractor for Urdu targets (general purpose, takes category_values) ---
def extract_category_from_target(text, category_values):
    if pd.isna(text):
        return None
    s = str(text).strip()

    # match any quote style including English, Urdu, curly, angled
    m = re.search(r'["“”«»\'‘’](.+?)["“”«»\'‘’]', s)
    if m:
        extracted = m.group(1).strip()
        if extracted in category_values:
            return extracted

    # match category by content if the category appears in the sentence
    for urdu_cat in category_values:
        if urdu_cat in s:
            return urdu_cat

    # fallback: return None if no explicit category found
    return None

# --- Clean predicted categories for News Classification ---
def clean_predicted_news_category(text, urdu_categories):
    if pd.isna(text):
        return None
    s = str(text).strip()

    # Look for the specific phrase "پریڈکٹیڈ زمرہ:" or "پریڈکٹڈ زمرہ:"
    # and extract the text after it.
    match = re.search(r'(پریڈکٹیڈ زمرہ:|پریڈکٹڈ زمرہ:)\s*(.*?)(?:\n|$)', s, re.DOTALL)
    if match:
        potential_category_text = match.group(2).strip()
    else:
        # Fallback heuristic: sometimes the model just outputs the category at the end.
        # This is less reliable but better than nothing if the tag isn't found.
        lines = s.split('\n')
        potential_category_text = lines[-1].strip() if lines else s.strip()

    # Remove any unwanted tokens like <pad>, </s>, etc.
    potential_category_text = potential_category_text.replace("<pad>", "").replace("</s>", "").strip()

    # Check for bolded categories first (e.g., **کھیل**)
    bold_match = re.search(r'\*\*([^\*]+)\*\*', potential_category_text)
    if bold_match:
        extracted_category = bold_match.group(1).strip()
        if extracted_category in urdu_categories:
            return extracted_category

    # Check for direct match or starts with a category
    for category in urdu_categories:
        if category == potential_category_text:
            return category
        if potential_category_text.startswith(category + " "): # Match category followed by a space
            return category
        if potential_category_text.startswith(category): # Direct match at start
            return category

    # If no specific tag or direct match, try to find any category as a substring
    # This is less precise but can catch some cases.
    for category in urdu_categories:
        if category in potential_category_text:
            return category

    # If no category is found, return None
    return None


# --- Final evaluator for News Classification ---
def evaluate_classification(df):

    df = df.copy()

    # Define news categories here to be explicitly used by this function
    news_category_values = [
        'پاکستان', 'کھیل', 'ٹیکنالوجی', 'تفریح', 'بین الاقوامی',
        'معیشت', 'طرز زندگی', 'رائے', 'تعلیم'
    ]

    # Extract actual categories from targets
    df["actual_cat"] = df["targets"].apply(lambda x: extract_category_from_target(x, news_category_values))

    # Clean predicted categories using the news-specific cleaner
    df["pred_cat"] = df["predicted_category"].apply(
        lambda x: clean_predicted_news_category(x, news_category_values)
    )

    # Drop rows where actual or predicted categories are None
    df = df[df["actual_cat"].notna() & df["pred_cat"].notna()].copy()

    # If no usable rows remain after filtering
    if df.empty:
        return {
            "accuracy": 0.0,
            "precision": 0.0,
            "recall": 0.0,
            "f1_score": 0.0,
            "auc_roc": 0.0, # Cannot compute AUC with no data
            "log_loss": 0.0 # Cannot compute LogLoss with no data
        }, pd.DataFrame(columns=df.columns) # Return an empty DataFrame with original columns

    y_true = df["actual_cat"]
    y_pred = df["pred_cat"]

    # Basic metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average="macro", zero_division=0)
    recall = recall_score(y_true, y_pred, average="macro", zero_division=0)
    f1 = f1_score(y_true, y_pred, average="macro", zero_division=0)

    # Prepare labels for AUC and LogLoss
    lb = LabelBinarizer()
    all_labels = pd.concat([y_true, y_pred]).unique()
    lb.fit(all_labels)

    y_true_b = lb.transform(y_true)
    y_pred_b = lb.transform(y_pred)

    auc_roc = 0.0
    ll = 0.0

    if len(lb.classes_) >= 2: # Need at least two classes for AUC and LogLoss
        y_pred_prob = y_pred_b.astype(float)
        eps = 1e-7 # Use a smaller epsilon for clip to avoid issues with probabilities
        y_pred_prob = np.clip(y_pred_prob, eps, 1 - eps)

        if len(lb.classes_) > 2: # Multiclass
            try:
                auc_roc = roc_auc_score(y_true_b, y_pred_prob, average="macro", multi_class="ovr")
            except ValueError:
                auc_roc = 0.0
        else: # Binary
            try:
                # If y_true_b is 2D (from LabelBinarizer on binary), take the column for the positive class.
                # roc_auc_score for binary expects 1D y_true and 1D y_score.
                if y_true_b.ndim > 1 and y_true_b.shape[1] == 1:
                    y_true_single_col = y_true_b.flatten()
                    y_pred_single_col = y_pred_prob.flatten()
                else:
                    y_true_single_col = y_true_b
                    y_pred_single_col = y_pred_prob

                auc_roc = roc_auc_score(y_true_single_col, y_pred_single_col, average="binary")
            except ValueError:
                auc_roc = 0.0

        try:
            ll = log_loss(y_true_b, y_pred_prob)
        except ValueError:
            ll = 0.0

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "auc_roc": auc_roc,
        "log_loss": ll
    }, df

In [61]:
zs_df = pd.read_parquet("model_zs_classification_results.parquet")
metrics, evaluated_df = evaluate_classification(zs_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

print(evaluated_df['actual_cat'].value_counts())
print(evaluated_df['pred_cat'].value_counts())



accuracy: 0.0000
precision: 0.0000
recall: 0.0000
f1_score: 0.0000
auc_roc: 0.0000
log_loss: 0.0000
actual_cat
تفریح    1
Name: count, dtype: int64
pred_cat
کھیل    1
Name: count, dtype: int64


In [62]:
ts_df = pd.read_parquet("model_ts_classification_results.parquet")
metrics, ts_df_evaluated = evaluate_classification(ts_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.4300
precision: 0.1075
recall: 0.2500
f1_score: 0.1503
auc_roc: 0.5000
log_loss: 9.1873


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [63]:
metrics, ts_df_evaluated = evaluate_classification(ts_df)


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [64]:
print(ts_df_evaluated['actual_cat'].value_counts())
print(ts_df_evaluated['pred_cat'].value_counts())

actual_cat
کھیل         43
تفریح        35
معیشت        15
ٹیکنالوجی     7
Name: count, dtype: int64
pred_cat
کھیل    100
Name: count, dtype: int64


In [65]:
# Use category_translation values to find category in predicted text
def standardize_predicted(text, urdu_categories):
    if pd.isna(text):
        return None
    s = str(text).strip()

    # check if any category appears in the sentence
    for c in urdu_categories:
        if c in s:
            return c

    # fallback: return None if no category is found
    return None


In [66]:
metrics, ts_df_evaluated = evaluate_classification(ts_df)

print(ts_df_evaluated['actual_cat'].value_counts())
print(ts_df_evaluated['pred_cat'].value_counts())


actual_cat
کھیل         43
تفریح        35
معیشت        15
ٹیکنالوجی     7
Name: count, dtype: int64
pred_cat
کھیل    100
Name: count, dtype: int64


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [67]:
def standardize_predicted(text, urdu_categories):
    if pd.isna(text):
        return None
    s = str(text).strip()

    # find the first matching category in the sentence
    for c in urdu_categories:
        if c in s:
            return c

    # fallback: return None if no category is found
    return None


In [68]:
metrics, ts_df_evaluated = evaluate_classification(ts_df)

print(ts_df_evaluated['actual_cat'].value_counts())
print(ts_df_evaluated['pred_cat'].value_counts())

for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

actual_cat
کھیل         43
تفریح        35
معیشت        15
ٹیکنالوجی     7
Name: count, dtype: int64
pred_cat
کھیل    100
Name: count, dtype: int64
accuracy: 0.4300
precision: 0.1075
recall: 0.2500
f1_score: 0.1503
auc_roc: 0.5000
log_loss: 9.1873


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [70]:
cot_df = pd.read_parquet("model_cot_classification_results.parquet")
metrics_dict, _ = evaluate_classification(cot_df)
for metric, value in metrics_dict.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.4300
precision: 0.1075
recall: 0.2500
f1_score: 0.1503
auc_roc: 0.5000
log_loss: 9.1873


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(


In [77]:
metrics, zs_df_evaluated = evaluate_classification(zs_df)

print(zs_df_evaluated['actual_cat'].value_counts())
print(zs_df_evaluated['pred_cat'].value_counts())

for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

actual_cat
تفریح    1
Name: count, dtype: int64
pred_cat
کھیل    1
Name: count, dtype: int64
accuracy: 0.0000
precision: 0.0000
recall: 0.0000
f1_score: 0.0000
auc_roc: 0.0000
log_loss: 0.0000


Sentiment

In [71]:
# Senitment

from datasets import load_dataset
import pandas as pd

# Load dataset (no second argument needed)
dataset = load_dataset('mteb/IndicSentiment' , 'ur')


# Convert "train" split to pandas
df = dataset["train"].to_pandas()

# Show first rows
df.head()


README.md: 0.00B [00:00, ?B/s]

test/ur.jsonl.gz:   0%|          | 0.00/236k [00:00<?, ?B/s]

train/ur.jsonl.gz:   0%|          | 0.00/37.3k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/156 [00:00<?, ? examples/s]

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang
0,Home,Appliances,Home theater,Soundbars,Boat,"Bluetooth/wireless, HDMI, audio output mode, i...",HDMI,This boat's soundbar is still wire-connectivit...,Negative,اس بوٹ کا ساؤنڈ بار تمام اسپیکرز کے لیے اب بھی...,ur
1,Hobbies,Music,Audio Output,headphones,Zeb Paradise,"on-ear, in-ear, wired, bluetooth, earbuds, noi...",Over-ear with mic,Foldable type of microphone with mic and micro...,Positive,مائیک اور مائیکرو ایس ڈی کارڈ سلاٹ کے ساتھ قاب...,ur
2,Entertainment,Apps,Social Media,Social networking,Instagram,"find friends, share photos and moments, free m...",daily status,The recently included feature of stories by de...,Positive,کہانیوں کی حالیہ خصوصیت بطور ڈیفالٹ 24 گھنٹے ت...,ur
3,Transportation,Air,Flights,International,Emirates,"luggage allowance, affordable rates, luxury, f...",Rates Luggage allowance,"Rates are competitive, almost always the best ...",Positive,"""قیمتیں مسابقت والی ہیں، مارکیٹ میں تقریباً ہم...",ur
4,Home,Appliances,Fan,Exhaust fan,Bajaj Maxima,"remove moisture/unpleasant odour, air delivery...",Front Shutter,Looks very big and efficient. But since there ...,Negative,بہت بڑا اور موثر ہے۔ لیکن چونکہ سامنے کوئی شٹر...,ur


In [72]:
zero_shot = """آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جائزے کے جذبات کا تعین کرنا ہے، جو مثبت یا منفی ہو سکتا ہے۔

"""


In [73]:
three_shot = """آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جائزے کے جذبات کا تعین کرنا ہے، جو مثبت یا منفی ہو سکتا ہے۔

### مثال ۱
### جائزہ:
قیمتیں بہت مناسب ہیں، تقریباً ہمیشہ مارکیٹ میں بہترین۔

### احساس:
مثبت

### مثال ۲
### جائزہ:
آن کال کنیکٹیویٹی اکثر بہت کمزور رہتی ہے۔

### احساس:
منفی

### مثال ۳
### جائزہ:
ہک اور لوپ ڈیزائن بہت آسانی سے سیٹ اپ اور لے جایا جا سکتا ہے۔

### آپ کا کام

"""


In [74]:
chain_of_thought = """آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جائزے کے جذبات کا تعین کرنا ہے، جو مثبت یا منفی ہو سکتا ہے۔ جذبات کا تعین کرتے وقت درج ذیل مراحل کی پیروی کریں:

### مرحلہ ۱: جائزہ پڑھیں
جائزے کو پڑھیں اور یہ سمجھنے کی کوشش کریں کہ اس میں کیا کہا گیا ہے۔

### مرحلہ ۲: مثبت یا منفی الفاظ کی نشاندہی
جائزے میں استعمال ہونے والے مثبت الفاظ (اچھا، مطمئن، شاندار) یا منفی الفاظ (برا، خراب، کم معیار) کی نشاندہی کریں۔

### مرحلہ ۳: جذبات کا تعین
اگر مثبت الفاظ زیادہ ہوں تو جذبات مثبت، اور اگر منفی الفاظ زیادہ ہوں تو جذبات منفی تصور کریں۔

### مثال ۱
### جائزہ:
قیمتیں مقابلے کی ہیں، تقریباً ہمیشہ مارکیٹ میں بہترین۔

#### سوچنے کا عمل:
- مثبت الفاظ: مقابلے کی، بہترین
- جذبات: مثبت

### جذبات:
مثبت

### مثال ۲
### جائزہ:
آن کال کنیکٹیویٹی اکثر بہت کمزور رہتی ہے۔

#### سوچنے کا عمل:
- منفی الفاظ: اکثر بہت کمزور
- جذبات: منفی

### جذبات:
منفی

### مثال ۳
### جائزہ:
ہک اور لوپ ڈیزائن بہت آسانی سے سیٹ اپ اور لے جایا جا سکتا ہے۔

#### سوچنے کا عمل:
- مثبت الفاظ: بہت آسانی سے سیٹ اپ اور لے جایا جا سکتا ہے
- جذبات: مثبت

### جذبات:
مثبت

### آپ کا کام


"""


In [75]:
category_translation = {
    'Positive': 'مثبت',
    'Negative': 'منفی',
}


In [76]:
import time
from tqdm.notebook import tqdm
import pandas as pd

# Use only first 100 rows for faster testing
df_subset = df.iloc[:100].copy()
df_subset['predicted_category'] = None

# Try to load progress if it exists
try:
    df_progress = pd.read_parquet('model_zs_urdu_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df_subset.copy()
    start_index = 0

# Loop over subset
for i, row in tqdm(df_subset.iloc[start_index:].iterrows(), initial=start_index, total=len(df_subset) - start_index):
    review = row['INDIC REVIEW']  # Urdu review text
    label = row['LABEL']          # Original label
    label_urdu = category_translation.get(label, label)

    # Zero-shot Urdu prompt
    prompt_input = zero_shot + " ###\nجائزہ: " + review + "\n### جذبات:"

    try:
        df_subset.at[i, 'predicted_category'] = get_model_output(prompt_input)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    # Save progress every 50 rows
    if (i + 1) % 50 == 0:
        df_subset.to_parquet('model_zs_urdu_sentiment_results.parquet')

# Save final results
df_subset.to_parquet('model_zs_urdu_sentiment_results.parquet')


  0%|          | 0/100 [00:00<?, ?it/s]

In [78]:
zs_results = pd.read_parquet("model_zs_urdu_sentiment_results.parquet")
zs_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category
95,Entertainment,Movies,Genres,Family Drama,The Parent Trap,"entertaining, family values, relatable charact...","Story-line, melodrama",At every scene where the parents appeared they...,Negative,ہر اس منظر میں جہاں والدین نمودار ہوئے، وہ اس ...,ur,آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جا...
96,Pets,Dogs,Grooming,Deodorizers,First Pet,"ayurvedic, scent, fresh, vet recommended","provides moisture to irritated, itchy, painful...",It is particularly effective in providing mois...,Positive,یہ خاص طور پر چڑچڑاپن، کھجلی اور تکلیف دہ جلدی...,ur,آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جا...
97,Hobbies,Music,Audio Input,Microphone,Rhinocables PC microphone,"frequency response, self-noise, sensitivity, p...","frequency response, self-noise,","quality is terrible,self noise exists",Negative,معیار خوفناک ہے، شور موجود ہے,ur,آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جا...
98,Hobbies,Photography,Camera accessories,Lens cap,Hanumex,"durability, dust and scratch resistent",Durability,Cap is manufactured with cheap quality plastic...,Negative,کیپ سستے معیاری پلاسٹک کے مواد سے تیار کی جاتی...,ur,آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جا...
99,Hobbies,Photography,Camera accessories,Lens case,Digicare,"thick material for protection, hook, compatibi...",thick material for protection,Neaprene material coating from outside but no ...,Negative,باہر سے نیوپرین مواد کی کوٹنگ لیکن اندر کوئی ک...,ur,آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جا...


In [79]:
import time
from tqdm.notebook import tqdm
import pandas as pd

# Use only first 100 rows for testing
df_subset = df.iloc[:100].copy()
df_subset['predicted_category'] = None

# Try to load progress if it exists
try:
    df_progress = pd.read_parquet('model_ts_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df_subset.copy()
    start_index = 0

# Loop over the subset
for i, row in tqdm(df_subset.iloc[start_index:].iterrows(), initial=start_index, total=len(df_subset) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_urdu = category_translation.get(label, label)  # Translate label if needed

    # Three-shot Urdu prompt
    prompt_input = three_shot + " ###\nجائزہ: " + review + "\n### جذبات:"

    try:
        df_subset.at[i, 'predicted_category'] = get_model_output(prompt_input, max_new_tokens=256)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    # Save progress every 50 rows
    if (i + 1) % 50 == 0:
        df_subset.to_parquet('model_ts_sentiment_results.parquet')

# Save final results
df_subset.to_parquet('model_ts_sentiment_results.parquet')


  0%|          | 0/100 [00:00<?, ?it/s]

In [80]:
ts_results = pd.read_parquet("model_ts_sentiment_results.parquet")
ts_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category
95,Entertainment,Movies,Genres,Family Drama,The Parent Trap,"entertaining, family values, relatable charact...","Story-line, melodrama",At every scene where the parents appeared they...,Negative,ہر اس منظر میں جہاں والدین نمودار ہوئے، وہ اس ...,ur,آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جا...
96,Pets,Dogs,Grooming,Deodorizers,First Pet,"ayurvedic, scent, fresh, vet recommended","provides moisture to irritated, itchy, painful...",It is particularly effective in providing mois...,Positive,یہ خاص طور پر چڑچڑاپن، کھجلی اور تکلیف دہ جلدی...,ur,آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جا...
97,Hobbies,Music,Audio Input,Microphone,Rhinocables PC microphone,"frequency response, self-noise, sensitivity, p...","frequency response, self-noise,","quality is terrible,self noise exists",Negative,معیار خوفناک ہے، شور موجود ہے,ur,آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جا...
98,Hobbies,Photography,Camera accessories,Lens cap,Hanumex,"durability, dust and scratch resistent",Durability,Cap is manufactured with cheap quality plastic...,Negative,کیپ سستے معیاری پلاسٹک کے مواد سے تیار کی جاتی...,ur,آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جا...
99,Hobbies,Photography,Camera accessories,Lens case,Digicare,"thick material for protection, hook, compatibi...",thick material for protection,Neaprene material coating from outside but no ...,Negative,باہر سے نیوپرین مواد کی کوٹنگ لیکن اندر کوئی ک...,ur,آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جا...


In [81]:
import time
from tqdm.notebook import tqdm
import pandas as pd

# Use only first 100 rows for testing
df_subset = df.iloc[:100].copy()
df_subset['predicted_category'] = None

# Try to load progress if it exists
try:
    df_progress = pd.read_parquet('model_cot_sentiment_results.parquet')
    start_index = df_progress['predicted_category'].last_valid_index() + 1
except (FileNotFoundError, ValueError):
    df_progress = df_subset.copy()
    start_index = 0

# Loop over the subset
for i, row in tqdm(df_subset.iloc[start_index:].iterrows(), initial=start_index, total=len(df_subset) - start_index):
    review = row['INDIC REVIEW']
    label = row['LABEL']
    label_urdu = category_translation.get(label, label)  # Translate label if needed

    # CoT Urdu prompt
    prompt_input = chain_of_thought + "\n### جائزہ: " + review

    try:
        df_subset.at[i, 'predicted_category'] = get_model_output(prompt_input, max_new_tokens=256)
    except Exception as e:
        print(f"Error at index {i}: {e}")
        time.sleep(5)

    # Save progress every 50 rows
    if (i + 1) % 50 == 0:
        df_subset.to_parquet('model_cot_sentiment_results.parquet')

# Save final results
df_subset.to_parquet('model_cot_sentiment_results.parquet')


  0%|          | 0/100 [00:00<?, ?it/s]

In [82]:
cot_results = pd.read_parquet("model_cot_sentiment_results.parquet")
cot_results.tail()

,GENERIC CATEGORIES,CATEGORY,SUB-CATEGORY,PRODUCT,BRAND,ASPECTS,ASPECT COMBO,ENGLISH REVIEW,LABEL,INDIC REVIEW,lang,predicted_category
95,Entertainment,Movies,Genres,Family Drama,The Parent Trap,"entertaining, family values, relatable charact...","Story-line, melodrama",At every scene where the parents appeared they...,Negative,ہر اس منظر میں جہاں والدین نمودار ہوئے، وہ اس ...,ur,آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جا...
96,Pets,Dogs,Grooming,Deodorizers,First Pet,"ayurvedic, scent, fresh, vet recommended","provides moisture to irritated, itchy, painful...",It is particularly effective in providing mois...,Positive,یہ خاص طور پر چڑچڑاپن، کھجلی اور تکلیف دہ جلدی...,ur,آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جا...
97,Hobbies,Music,Audio Input,Microphone,Rhinocables PC microphone,"frequency response, self-noise, sensitivity, p...","frequency response, self-noise,","quality is terrible,self noise exists",Negative,معیار خوفناک ہے، شور موجود ہے,ur,آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جا...
98,Hobbies,Photography,Camera accessories,Lens cap,Hanumex,"durability, dust and scratch resistent",Durability,Cap is manufactured with cheap quality plastic...,Negative,کیپ سستے معیاری پلاسٹک کے مواد سے تیار کی جاتی...,ur,آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جا...
99,Hobbies,Photography,Camera accessories,Lens case,Digicare,"thick material for protection, hook, compatibi...",thick material for protection,Neaprene material coating from outside but no ...,Negative,باہر سے نیوپرین مواد کی کوٹنگ لیکن اندر کوئی ک...,ur,آپ کو ایک پروڈکٹ کا جائزہ دیا گیا ہے۔ آپ کو جا...


In [83]:
def clean_predicted_category(text, category_translation):
    if text is None:
        return ""

    # Assuming `text` contains the original prompt + model's generation.
    # The generation for the category should typically appear after the last 'جذبات:' tag.
    import re

    # Find the last occurrence of 'جذبات:' in the text
    last_sentiment_tag_index = text.rfind("جذبات:")
    if last_sentiment_tag_index != -1:
        # Extract the content after the last 'جذبات:'
        potential_category_text = text[last_sentiment_tag_index + len("جذبات:"):].strip()
    else:
        # If 'جذبات:' is not found, consider the entire text for pattern matching
        potential_category_text = text

    # First, check if any category is directly present in the extracted text
    for category_urdu in category_translation.values():
        if category_urdu in potential_category_text:
            return category_urdu

    # If no direct match, check for text within ** ** (e.g., '**مثبت**')
    match = re.search(r'\*\*([^*]*)\*\*', potential_category_text)
    if match:
        cleaned_text = match.group(1).strip() # Extract and strip whitespace
        for category_urdu in category_translation.values():
            if cleaned_text == category_urdu:
                return category_urdu

    # If no category is found or extracted, return an empty string
    return ""

In [ ]:
clean_predicted_category("جذبات: **منفی** \n\nجائزہ میں کہا گیا ہے ...", category_translation)


In [84]:
clean_predicted_category("جذبات: **مثبت** \n\nقیمتیں مقابلے کی ہیں...", category_translation)


'مثبت'

In [85]:
clean_predicted_category(None, category_translation)

''

In [86]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
from sklearn.preprocessing import LabelBinarizer

def update_cleaned_predicted_category(row):
    if pd.isna(row['cleaned_predicted_category']) or row['cleaned_predicted_category'] == '':
        if row['category_ur'] == 'مثبت':
            return 'منفی'
        elif row['category_ur'] == 'منفی':
            return 'مثبت'
    return row['cleaned_predicted_category']

def evaluate_classification(df):
    df['category'] = df['LABEL']
    # Map English category to Urdu
    df['category_ur'] = df['category'].map(category_translation)

    # Clean the predicted categories
    df['cleaned_predicted_category'] = df['predicted_category'].apply(clean_predicted_category, args=(category_translation,))

    # Update the cleaned predicted categories
    for index, row in df.iterrows():
        old_value = row['cleaned_predicted_category']
        if pd.isna(old_value) or old_value == '':
            if row['category_ur'] == 'مثبت':
                new_value = 'منفی'
            elif row['category_ur'] == 'منفی':
                new_value = 'مثبت'
            df.at[index, 'cleaned_predicted_category'] = new_value

    unique_labels = set(df['category_ur'].unique()).union(set(df['cleaned_predicted_category'].unique()))
    if len(unique_labels) > 2:
        raise ValueError(f"Expected binary classification but found multiple labels: {unique_labels}")

    # Calculate metrics
    y_true = df['category_ur']
    y_pred = df['cleaned_predicted_category']

    # Ensure the labels are binary
    lb = LabelBinarizer()
    y_true_binarized = lb.fit_transform(y_true)
    y_pred_binarized = lb.transform(y_pred)

    if len(lb.classes_) != 2:
        raise ValueError(f"Expected binary classification but found these classes: {lb.classes_}")

    pos_label = lb.classes_[1]  # The positive label

    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, pos_label=pos_label, average='binary', zero_division=0)
    recall = recall_score(y_true, y_pred, pos_label=pos_label, average='binary', zero_division=0)
    f1 = f1_score(y_true, y_pred, pos_label=pos_label, average='binary', zero_division=0)
    auc_roc = roc_auc_score(y_true_binarized, y_pred_binarized)
    logloss = log_loss(y_true_binarized, y_pred_binarized)

    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'auc_roc': auc_roc,
        'log_loss': logloss
    }

    return metrics


In [87]:
zs_df = pd.read_parquet("model_zs_urdu_sentiment_results.parquet")
metrics = evaluate_classification(zs_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.0500
precision: 0.0238
recall: 0.0182
f1_score: 0.0206
auc_roc: 0.0535
log_loss: 34.2415


In [88]:
ts_df = pd.read_parquet("model_ts_sentiment_results.parquet")
metrics = evaluate_classification(ts_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.2400
precision: 0.2558
recall: 0.2000
f1_score: 0.2245
auc_roc: 0.2444
log_loss: 27.3932


In [89]:
cot_df = pd.read_parquet("model_cot_sentiment_results.parquet")
metrics = evaluate_classification(cot_df)
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

accuracy: 0.3900
precision: 0.4211
recall: 0.2909
f1_score: 0.3441
auc_roc: 0.4010
log_loss: 21.9866
